# Capstone Project
## Applied Data Science Capstone by IBM

## Table of contents
* [Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Business Problem <a name="introduction"></a>

Anyone, who wants to open a venue in a certain city is faced with the question of "Where should I open my venue?". Obviously, location is very important as it is usually the main driver in retail and gastronomy. 

Imagine I am a owner of a coffee shop franchise and I would like to understand in what suburb should I open a new coffee shop in Vancouver. My question is the following:
- In what suburb should I open a new coffee shop?

This problem is relevant for any person interested in opening a venue in a certain city. The methodology taken for the rest of the project, can easily be modified to work for any location and any venue category listed on Foursquare. 

## Data <a name="data"></a>

#### What data will be used?

In order to answer the question above, I will need information about existing venues and I will need some information about Vancouver's suburbs. In particular, it is not enough to simply look at the number of coffee shops and a suburb and decide on the suburb with the lowest number of coffee shops. Why?

Maybe the suburb with the lowest number of coffee shops has the highest unemployment rate, so disposable income in that are will be quite restricted. We need to understand what variables or characteristics of the suburbs have a statistically significant impact on the number of coffee shops. 

Therefore, we need two data sets:
    
- <b>Census data: </b>Census data for Vancouver is available here: <br>
    https://opendata.vancouver.ca/explore/dataset/census-local-area-profiles-2016/information/ <br><br>Unfortunately the census data is from 2016, but we need to assume for this exercise that the demographics have not changed until today.
The census data lists 22 suburbs (or local areas) and provides demographic information about the people, who are living there. 
For example, there are 4000 people between the age of 0 and 14 living in 'Downtown' at the time of the census. 
The data can freely be downloaded in a csv or xls format. <br><br>

- <b> FourSquare data:</b> Using the API from FourSquare allows us to get information about venues (amongst other things) and use that information to identify any trends or     clusters. 

## Methodology <a name="methodology"></a>
1. Load publicly available census data for Vancouver
2. Extract only relevant variables from the census data that could have a significant impact on the number of coffee shops (e.g., language spoken at home is deemed to have minimal impact on the number of coffee shops in a suburb)
3. Derive the number of coffee shops for each suburb in Vancouver through using the Foursquare API
4. Merge the two different data sets
5. Run a correlation analysis against all variables
6. Understand if any of the calculated correlation coefficients are statistically significant with p<=0.05
7. Results: Identify suburb based on the values of the statistically significant variables<br><br>
The suburb identified in step 7, will be the one that is proposed to be analysed further for opening a new coffee shop development.

## Analysis <a name="analysis"></a>
### Load publicly available census data for Vancouver

In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
# import Vancouver census data

loc = pd.read_csv("https://webtransfer.vancouver.ca/opendata/csv/CensusLocalAreaProfiles2016.csv",encoding = "ISO-8859-1", skiprows=4)

In [3]:
loc.shape

(5589, 26)

In [4]:
# remove empty rows

loc2 = loc[loc['Variable'].notna()]
loc2.reset_index(drop=True)

,ID,Variable,Arbutus-Ridge,Downtown,Dunbar-Southlands,Fairview,Grandview-Woodland,Hastings-Sunrise,Kensington-Cedar Cottage,Kerrisdale,...,Riley Park,Shaughnessy,South Cambie,Strathcona,Sunset,Victoria-Fraserview,West End,West Point Grey,Vancouver CSD,Vancouver CMA
0,1,Total - Age groups and average age of the pop...,"15,295","62,030","21,425","33,620","29,175","34,575","49,325","13,975",...,"22,555","8,430","7,970","12,585","36,500","31,065","47,200","13,065","631,485","2,463,430"
1,2,0 to 14 years,2015,4000,3545,2580,3210,4595,7060,1880,...,3415,1175,1105,1065,5460,3790,1945,1900,70530,362110
2,3,0 to 4 years,455,2080,675,1240,1320,1510,2515,430,...,1175,270,360,360,1695,1175,965,420,24615,116270
3,4,5 to 9 years,685,1105,1225,760,1025,1560,2390,600,...,1160,405,365,365,1780,1210,560,670,22905,122730
4,5,10 to 14 years,880,810,1650,580,865,1525,2160,845,...,1080,500,375,340,1985,1410,415,810,23010,123110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5488,5489,Non-Aboriginal,360,1300,335,505,305,750,1125,360,...,350,120,115,125,850,945,915,215,13300,46615
5489,5490,English and French,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,15
5490,5491,English and non-official language,10,10,0,0,0,15,20,10,...,0,0,0,0,10,30,0,0,190,580
5491,5492,French and non-official language,0,15,10,20,10,0,20,0,...,10,15,10,0,0,15,20,10,265,685


In [5]:
#transpose the dataframe, so 

loc2.set_index('ID',inplace=True)
loc3 = loc2.transpose()

### Extract only relevant variables from the census data

Based on the Census information, I have decided to choose the number of people living in each suburb ('population', the average age of the respective population in each suburb ('avg_age'), as well as median age('med_age'), average household size ('avg_household_size'), average income ('avg_income'), median income ('med_income'), employment rate ('employment_rate') and the number of people commuting to other suburbs ('commuters') as variables that could potential impact on the number of coffee shops in a suburb. 

In [6]:
#copy relevant variables to new dataframe
loc4 = loc3[['1','94','95','157','1858','1859','2142','2143','2300']].copy() 
#rename columns to something meaningful based on first row
loc4.columns=['population','avg_age','med_age','avg_household_size','avg_income','med_income','employment_rate','unemployment_rate','commuters']
#drop first row 
loc5 = loc4.drop(loc4.index[0])
#drop the last 2 rows as they are not suburbs of Vancouver
vancouver_data = loc5.iloc[:-2]
vancouver_data = vancouver_data.reset_index()
vancouver_data.rename(columns={'index':'suburbs'}, inplace=True)
vancouver_data["population"] = vancouver_data["population"].str.replace(",", "")
vancouver_data

,suburbs,population,avg_age,med_age,avg_household_size,avg_income,med_income,employment_rate,unemployment_rate,commuters
0,Arbutus-Ridge,15295,44.6,46.2,2.4,62675,30929,48.4,6.6,1400
1,Downtown,62030,40.6,37.1,1.7,63251,41858,68.8,5.6,7010
2,Dunbar-Southlands,21425,41.1,44.1,2.8,78117,40463,53.9,6.2,2370
3,Fairview,33620,43.4,40.2,1.7,61627,46940,71.8,4.6,4675
4,Grandview-Woodland,29175,40.2,38.1,1.9,42896,32438,69.9,5.3,4085
5,Hastings-Sunrise,34575,42.3,42.1,2.7,38258,27255,60.4,5.9,5180
6,Kensington-Cedar Cottage,49325,40,38.8,2.7,38411,28356,65.1,5.9,7500
7,Kerrisdale,13975,42.9,45.6,2.5,77248,35064,49,7.5,1330
8,Killarney,29325,42.4,43.4,2.7,39013,29259,59.1,5.4,5325
9,Kitsilano,43045,40.6,37.7,1.9,63092,44084,71.1,5.2,5665


### Derive the number of coffee shops for each suburb
As a first step, we need to get the longitude and latitude for each suburb, which will then be used in our requests to the FourSquare API.


In [7]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [8]:
#get all suburb names in a list
suburbs_list = vancouver_data['suburbs'].tolist()
print(f"\nVancouver Suburbs List:\n{suburbs_list}\ntype:{type(suburbs_list)}")


Vancouver Suburbs List:
['Arbutus-Ridge ', 'Downtown ', 'Dunbar-Southlands ', 'Fairview ', 'Grandview-Woodland ', 'Hastings-Sunrise ', 'Kensington-Cedar Cottage ', 'Kerrisdale ', 'Killarney ', 'Kitsilano ', 'Marpole ', 'Mount Pleasant ', 'Oakridge ', 'Renfrew-Collingwood ', 'Riley Park ', 'Shaughnessy ', 'South Cambie ', 'Strathcona ', 'Sunset ', 'Victoria-Fraserview', 'West End ', 'West Point Grey ']
type:<class 'list'>


In [9]:
#get langitude and longitude for each suburb and write in a list
geolocator = Nominatim(user_agent="Vancouver_explorer")

city = 'Vancouver, Canada'
rows = []
for suburb in suburbs_list:
    address = suburb
    location = geolocator.geocode(address)
    lat = location.latitude
    lng = location.longitude
    rows.append([address, lat, lng])

In [10]:
#convert list to dataframe
geo_data = pd.DataFrame(rows, columns=["suburbs", "latitude","longitude"])

geo_data

,suburbs,latitude,longitude
0,Arbutus-Ridge,49.246305,-123.159636
1,Downtown,34.042849,-118.247673
2,Dunbar-Southlands,49.237864,-123.184354
3,Fairview,40.633372,-90.163732
4,Grandview-Woodland,49.275849,-123.066934
5,Hastings-Sunrise,49.278714,-123.039998
6,Kensington-Cedar Cottage,49.246790,-123.073475
7,Kerrisdale,49.220985,-123.159548
8,Killarney,52.059575,-9.505322
9,Kitsilano,49.269410,-123.155267


### Get Foursquare data

In [11]:
import json # library to handle JSON files
import requests # library to handle requests

In [12]:
#exploring Toronto through Foursquare
CLIENT_ID = 'SRMPT4L3BCR5EJFDHVC2BO3OQ555NGCFEIZYO4PZHDHTD1KB' # your Foursquare ID
CLIENT_SECRET = '0REPJIWMIAUK25KRSVJXNQSYE1XFJSPFRCQ3C4DDMBL3GRR4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SRMPT4L3BCR5EJFDHVC2BO3OQ555NGCFEIZYO4PZHDHTD1KB
CLIENT_SECRET:0REPJIWMIAUK25KRSVJXNQSYE1XFJSPFRCQ3C4DDMBL3GRR4


In [13]:
# get venues from suburbs

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
vancouver_venues = getNearbyVenues(names=geo_data['suburbs'],
                                   latitudes=geo_data['latitude'],
                                   longitudes=geo_data['longitude']
                                  )

Arbutus-Ridge 
Downtown 
Dunbar-Southlands 
Fairview 
Grandview-Woodland 
Hastings-Sunrise 
Kensington-Cedar Cottage 
Kerrisdale 
Killarney 
Kitsilano 
Marpole 
Mount Pleasant 
Oakridge 
Renfrew-Collingwood 
Riley Park 
Shaughnessy 
South Cambie 
Strathcona 
Sunset 
Victoria-Fraserview
West End 
West Point Grey 


In [15]:
print(vancouver_venues.shape)
vancouver_venues.head()

(420, 7)


,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arbutus-Ridge,49.246305,-123.159636,Starbucks,49.244768,-123.153891,Coffee Shop
1,Arbutus-Ridge,49.246305,-123.159636,Subway,49.244558,-123.153975,Sandwich Place
2,Arbutus-Ridge,49.246305,-123.159636,Dollarama,49.248885,-123.154049,Discount Store
3,Arbutus-Ridge,49.246305,-123.159636,Dance Co,49.248822,-123.154979,Dance Studio
4,Arbutus-Ridge,49.246305,-123.159636,M&M Food Market,49.244695,-123.153883,Grocery Store


In [16]:
# condense to number of neighborhoods
vancouver_venues.groupby('Suburb').count()

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Arbutus-Ridge,10,10,10,10,10,10
Downtown,47,47,47,47,47,47
Dunbar-Southlands,12,12,12,12,12,12
Fairview,4,4,4,4,4,4
Grandview-Woodland,42,42,42,42,42,42
Hastings-Sunrise,23,23,23,23,23,23
Kensington-Cedar Cottage,15,15,15,15,15,15
Kerrisdale,4,4,4,4,4,4
Killarney,52,52,52,52,52,52


### Analyse suburbs

In [17]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vancouver_onehot['Suburb'] = vancouver_venues['Suburb'] 

In [18]:
# move suburb column to the first column
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

,Suburb,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,...,Theater,Theme Park,Theme Park Ride / Attraction,Tiki Bar,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#group by neighborhoods and use the sum to calculate the merged scores - how many vanues are there from each type in a certain suburb

vancouver_grouped = vancouver_onehot.groupby('Suburb').sum().reset_index()
pd.set_option("max_columns", None)
vancouver_grouped

,Suburb,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Escape Room,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hookah Bar,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Inn,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Record Shop,Rest Area,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tiki Bar,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Arbutus-Ridge,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Downtown,2,0,2,0,0,0,0,0,0,5,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,0,0,3,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
2,Dunbar-Southlands,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fairview,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Grandview-Woodland,0,0,0,1,1,1,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,4,0,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,4,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,2,0,0,0,1,2,0,0,0,0
5,Hastings-Sunrise,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,4,0,0,0,1,0,0,0
6,Kensington-Cedar Cottage,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [20]:
#copy cafes and coffee shops into new dataframe
vancouver_c=vancouver_grouped[['Suburb', 'Coffee Shop']].copy()
vancouver_c.rename(columns={'Suburb':'suburbs'}, inplace=True)
vancouver_c

,suburbs,Coffee Shop
0,Arbutus-Ridge,1
1,Downtown,3
2,Dunbar-Southlands,1
3,Fairview,0
4,Grandview-Woodland,4
5,Hastings-Sunrise,0
6,Kensington-Cedar Cottage,0
7,Kerrisdale,0
8,Killarney,1
9,Kitsilano,2


Based on the above table we can see that 'South Crambie' and 'Grandview-Woodland' have the highest number of Coffee Shops in Vancouver.

### Merge the two different data sets

In [21]:
#merge data sets
vancouver_cc = pd.merge(vancouver_c, vancouver_data, on="suburbs")
vancouver_cc

,suburbs,Coffee Shop,population,avg_age,med_age,avg_household_size,avg_income,med_income,employment_rate,unemployment_rate,commuters
0,Arbutus-Ridge,1,15295,44.6,46.2,2.4,62675,30929,48.4,6.6,1400
1,Downtown,3,62030,40.6,37.1,1.7,63251,41858,68.8,5.6,7010
2,Dunbar-Southlands,1,21425,41.1,44.1,2.8,78117,40463,53.9,6.2,2370
3,Fairview,0,33620,43.4,40.2,1.7,61627,46940,71.8,4.6,4675
4,Grandview-Woodland,4,29175,40.2,38.1,1.9,42896,32438,69.9,5.3,4085
5,Hastings-Sunrise,0,34575,42.3,42.1,2.7,38258,27255,60.4,5.9,5180
6,Kensington-Cedar Cottage,0,49325,40,38.8,2.7,38411,28356,65.1,5.9,7500
7,Kerrisdale,0,13975,42.9,45.6,2.5,77248,35064,49,7.5,1330
8,Killarney,1,29325,42.4,43.4,2.7,39013,29259,59.1,5.4,5325
9,Kitsilano,2,43045,40.6,37.7,1.9,63092,44084,71.1,5.2,5665


In [22]:
#convert data type objects to numeric

cols = vancouver_cc.columns.drop('suburbs')

vancouver_cc[cols] = vancouver_cc[cols].apply(pd.to_numeric, errors='coerce')

### Run a correlation analysis against all variables

In [23]:
#calculate the correlation between each of the variables to understand how each of the variables influence the other one

vancouver_cc.corr(method='pearson')

,Coffee Shop,population,avg_age,med_age,avg_household_size,avg_income,med_income,employment_rate,unemployment_rate,commuters
Coffee Shop,1.000000,0.062829,-0.384844,-0.488890,-0.383233,0.041327,0.365074,0.454483,-0.162733,-0.038293
population,0.062829,1.000000,-0.551671,-0.689048,-0.143606,-0.413710,-0.020088,0.603686,-0.370377,0.951421
avg_age,-0.384844,-0.551671,1.000000,0.875572,-0.015517,0.066995,-0.350812,-0.752967,0.571915,-0.553665
med_age,-0.488890,-0.689048,0.875572,1.000000,0.293920,0.208184,-0.369622,-0.931352,0.616820,-0.651081
avg_household_size,-0.383233,-0.143606,-0.015517,0.293920,1.000000,-0.007991,-0.330467,-0.411232,-0.026252,0.028195
avg_income,0.041327,-0.413710,0.066995,0.208184,-0.007991,1.000000,0.752169,-0.105028,-0.187255,-0.523414
med_income,0.365074,-0.020088,-0.350812,-0.369622,-0.330467,0.752169,1.000000,0.494280,-0.520051,-0.145513
employment_rate,0.454483,0.603686,-0.752967,-0.931352,-0.411232,-0.105028,0.494280,1.000000,-0.677841,0.574695
unemployment_rate,-0.162733,-0.370377,0.571915,0.616820,-0.026252,-0.187255,-0.520051,-0.677841,1.000000,-0.378089
commuters,-0.038293,0.951421,-0.553665,-0.651081,0.028195,-0.523414,-0.145513,0.574695,-0.378089,1.000000


### Understand if any of the calculated correlation coefficients are statistically significant with p<=0.05

In [24]:
#import library to calculate pvalues

from scipy.stats import kendalltau, pearsonr, spearmanr
    
def pearsonr_pval(x,y):
        return pearsonr(x,y)[1]

In [25]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

pvalues = vancouver_cc.corr(method=pearsonr_pval)
pvalues

,Coffee Shop,population,avg_age,med_age,avg_household_size,avg_income,med_income,employment_rate,unemployment_rate,commuters
Coffee Shop,1.000,0.787,0.085,0.025,0.086,0.859,0.104,0.038,0.481,0.869
population,0.787,1.000,0.010,0.001,0.535,0.062,0.931,0.004,0.098,0.000
avg_age,0.085,0.010,1.000,0.000,0.947,0.773,0.119,0.000,0.007,0.009
med_age,0.025,0.001,0.000,1.000,0.196,0.365,0.099,0.000,0.003,0.001
avg_household_size,0.086,0.535,0.947,0.196,1.000,0.973,0.143,0.064,0.910,0.903
avg_income,0.859,0.062,0.773,0.365,0.973,1.000,0.000,0.650,0.416,0.015
med_income,0.104,0.931,0.119,0.099,0.143,0.000,1.000,0.023,0.016,0.529
employment_rate,0.038,0.004,0.000,0.000,0.064,0.650,0.023,1.000,0.001,0.006
unemployment_rate,0.481,0.098,0.007,0.003,0.910,0.416,0.016,0.001,1.000,0.091
commuters,0.869,0.000,0.009,0.001,0.903,0.015,0.529,0.006,0.091,1.000


Based on the above table we can see that the p-values for median age ('med_age') and employment rate ('employment_rate') are statistically significant (p<=0.05). Therefore, the lower the median age in a suburb and the more people are employed, the more number of coffee shops can be found in a suburb!

## Results and Discussion <a name="results"></a>

Now that we know that median age and the employment rate are key factor influencing the number of coffee shops in a suburb, we need to use these results and identify which suburb we should investigate further. 

In [26]:
vancouver_cc.sort_values(by=['med_age'], inplace=True)
vancouver_cc

,suburbs,Coffee Shop,population,avg_age,med_age,avg_household_size,avg_income,med_income,employment_rate,unemployment_rate,commuters
11,Mount Pleasant,1,32955,38.300,35.500,1.800,54260,42362,77.900,4.700,4750
1,Downtown,3,62030,40.600,37.100,1.700,63251,41858,68.800,5.600,7010
9,Kitsilano,2,43045,40.600,37.700,1.900,63092,44084,71.100,5.200,5665
4,Grandview-Woodland,4,29175,40.200,38.100,1.900,42896,32438,69.900,5.300,4085
18,Sunset,0,36500,39.800,38.700,3.100,34212,25498,62.300,5.200,5635
6,Kensington-Cedar Cottage,0,49325,40.000,38.800,2.700,38411,28356,65.100,5.900,7500
14,Riley Park,3,22555,40.200,39.300,2.500,53060,37327,66.200,4.900,2825
13,Renfrew-Collingwood,0,51530,41.200,40.000,2.700,33360,25476,61.000,5.800,9070
3,Fairview,0,33620,43.400,40.200,1.700,61627,46940,71.800,4.600,4675
16,South Cambie,4,7970,42.100,40.200,2.400,65459,42094,63.300,6.700,855


Sorting our merged table based on median age, we find that Mount Pleasant has the lowest median age of all suburbs and the employment rate is higher than in 'South Cambie' and 'Grandview-Woodland'. Hence, I would choose 'Mount Pleasant' to investigate further with regards to the suitability of establishing a new coffee shop. 

Next steps would be to look at available properties and overall costs. This is required as the above analysis did not cover all variables that play a role in deciding if establishing a new coffee shop is a profitable endeavour in a certain suburb or not. I can, however, indicatively point to factors that have a statistically significant impact on the number of coffee shops in a suburb, which can and should be used as a starting point for further analysis.

## Conclusion <a name="conclusion"></a>

This project set out to apply lessons learned throughout all stages of the data science methodology from defining the business problem, understanding the data that will be used, the methodology, analysing the results and discussing the practical application and next step.

In this particular project, I wanted to know if combining census data and location data can provide any meaningful insights into how population characteristics in Vancouver's suburbs impact on the number of coffee shops. This has been successfully achieved.